In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention

In [2]:
# Input sequences (numbers)
input_texts = ["123", "456", "789"]

# Output sequences (words)
target_texts = ["one two three", "four five six", "seven eight nine"]

# Create vocabularies for input and output
input_vocab = sorted(set("".join(input_texts)))  # Characters: ['1', '2', ..., '9']
output_vocab = sorted(set(" ".join(target_texts)))  # Words: ['e', 'f', ..., 'v', 'x']

# Map each character/word to an index
input_token_index = {char: i for i, char in enumerate(input_vocab)}
output_token_index = {char: i for i, char in enumerate(output_vocab)}
reverse_output_token_index = {i: char for char, i in output_token_index.items()}

# Maximum sequence lengths
max_encoder_seq_length = max(len(text) for text in input_texts)
max_decoder_seq_length = max(len(text) for text in target_texts)

# Number of unique tokens
num_encoder_tokens = len(input_vocab)
num_decoder_tokens = len(output_vocab)

# One-hot encode input and target sequences
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

# Fill the one-hot encoded arrays
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, output_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, output_token_index[char]] = 1.0

## Build the Encoder-Decoder Model with Attention

In [3]:
# Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))  # Input shape: (sequence_length, num_encoder_tokens)
encoder_lstm = LSTM(256, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)  # Hidden states for attention

# Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))  # Input shape: (sequence_length, num_decoder_tokens)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])

# Attention layer
attention = Attention()
context_vector = attention([decoder_outputs, encoder_outputs])

# Concatenate context vector with decoder LSTM output
decoder_combined_context = Dense(256, activation="tanh")(context_vector)

# Final output layer (softmax activation for probabilities)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_combined_context)

# Full model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None, 9)        │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, None, 15)       │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, None, 256),    │        272,384 │ input_layer[0][0]      │
│                           │ (None, 256), (None,    │                │                        │
│                           │ 256)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, None, 256),    │        278,528 │ input_layer_1[0][0],   │
│                           │ (None, 256), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 256)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention (Attention)     │ (None, None, 256)      │              0 │ lstm_1[0][0],          │
│                           │                        │                │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, None, 256)      │         65,792 │ attention[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, None, 15)       │          3,855 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 620,559 (2.37 MB)

 Trainable params: 620,559 (2.37 MB)

 Non-trainable params: 0 (0.00 B)

## Train the Model

In [4]:
model.fit(
    [encoder_input_data, decoder_input_data],  # Input sequences and target sequences
    decoder_target_data,                       # Target sequence shifted by one time step
    batch_size=64,
    epochs=50,
    validation_split=0.2,
)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.2500 - loss: 2.0269 - val_accuracy: 0.1875 - val_loss: 2.5286
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.2812 - loss: 2.0142 - val_accuracy: 0.1875 - val_loss: 2.5230
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.2812 - loss: 2.0013 - val_accuracy: 0.1875 - val_loss: 2.5172
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.2812 - loss: 1.9880 - val_accuracy: 0.1875 - val_loss: 2.5111
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.2812 - loss: 1.9740 - val_accuracy: 0.1875 - val_loss: 2.5043
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.2812 - loss: 1.9591 - val_accuracy: 0.1875 - val_loss: 2.4970
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.2812 - loss: 1.9432 - val_accuracy: 0.1875 - val_loss: 2.4888
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.2812 - loss: 1.9261 - val_accuracy: 0.1875 - val_loss: 2.479

## Create Inference Models for Prediction

In [5]:
# Encoder Model
encoder_model = Model(encoder_inputs, [encoder_outputs, state_h, state_c])

# Decoder Model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_hidden_states_input = Input(shape=(None, 256))

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=[decoder_state_input_h, decoder_state_input_c]
)
context_vector = attention([decoder_outputs, decoder_hidden_states_input])
decoder_combined_context = Dense(256, activation="tanh")(context_vector)
decoder_outputs = decoder_dense(decoder_combined_context)

decoder_model = Model(
    [decoder_inputs, decoder_hidden_states_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs, state_h, state_c],
)

## Predict the Output Sequence

In [6]:
def decode_sequence(input_seq):
    # Encode the input sequence
    encoder_outputs, state_h, state_c = encoder_model.predict(input_seq)

    # Start the target sequence with the "start" token
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, output_token_index[" "]] = 1.0  # Assume " " is the start token

    decoded_sentence = ""
    stop_condition = False

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq, encoder_outputs, state_h, state_c]
        )

        # Get the most likely token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_output_token_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Stop if end token is reached or max length is exceeded
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update target sequence and states
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        state_h, state_c = h, c

    return decoded_sentence

## Test the Model

In [7]:
# Test input sequence
test_input = "123"
input_seq = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype="float32")
for t, char in enumerate(test_input):
    input_seq[0, t, input_token_index[char]] = 1.0

# Decode the sequence
decoded_sentence = decode_sequence(input_seq)
print(f"Input: {test_input}")
print(f"Output: {decoded_sentence}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Input: 123
Output: wwwwwwwwwwwwwwwww
